In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import tldextract
import urllib

from bs4 import BeautifulSoup

import sys
sys.path.append("/scratch/venia/web2wiki/helpers")


In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa

from bs4 import BeautifulSoup

import os




os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"

config = pyspark.SparkConf().setAll([('spark.executor.memory', '50g'),
                                 ('spark.executor.cores', '40'),
                                 ('spark.driver.memory','50g'),
                                 ('spark.driver.maxResultSize','0'),
                                 ('spark.python.worker.memory', '5g'),
                                 ('spark.reducer.maxSizeInFlight','5g'),
                                 ('spark.rpc.message.maxSize', '1000'),
                                 ('spark.sql.autoBroadcastJoinThreshold','-1'),
                                 ('spark.local.dir', '/tmp/')])
sc = pyspark.SparkContext(conf=config)
spark = SparkSession.builder.appName('venia2').config(conf=config).getOrCreate()



22/09/06 13:40:29 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/09/06 13:40:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/06 13:40:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/06 13:40:30 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/09/06 13:40:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/06 13:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/06 13:4

In [8]:
data = spark.read.load("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/wiki_content.parquet")


In [10]:
d = data.limit(1000).toPandas()

In [19]:
d.iloc[0]

url                                    http://blog.eciton.net
a1                                                          p
a2                                                        div
wiki_url      http://en.wikipedia.org/wiki/Master_boot_record
text1       <p>Apart from this feeling a bit ... cargo-cul...
text2       <div>\n<h1><a href="http://blog.eciton.net/uef...
Name: 0, dtype: object

In [13]:
import justext


In [22]:
paragraphs = justext.justext(d.text2.iloc[50], justext.get_stoplist("English"))


In [23]:
d.text2.iloc[50]

'<ul>\n<li><a href="http://en.wikipedia.org/wiki/2012" rel="nofollow" title="2012">2012 (Wikipedia EN)</a></li>\n<li><a href="http://bg.wikipedia.org/wiki/2012" rel="nofollow" title="2012">2012 (Wikipedia BG)</a></li>\n</ul>'

In [24]:
for paragraph in paragraphs:
    if paragraph.is_boilerplate:
        print(paragraph.text)

2012 (Wikipedia EN)
2012 (Wikipedia BG)


In [3]:
import glob
files = np.random.choice(glob.glob("/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/*"), 100)

In [6]:
list(files)

['/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210307231028-20210308021028-00392.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210308021603-20210308051603-00421.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210305060756-20210305090756-00029.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210302095427-20210302125427-00138.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210307170119-20210307200119-00477.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210228205741-20210228235741-00467.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210225095141-20210225125141-00585.warc.gz.parquet',
 '/dlabdata1/dlab_common_datasets/web2wiki/FullHTMLFeb2021/CC-MAIN-20210302003534-20210302033534-00565.warc.gz.parquet',
 '/dlabdata1/dlab_common_dataset

In [7]:
df = spark.read.load(list(files))

In [ ]:
df.count()